# Coummunity Detection for LA


Visualize the graph community for diverse time.

[weekdays]<br>
1. commuted time<br>
    6-10<br>
    16-21<br>
2. regular time(except commuted time)<br>
    10-16<br>
    
[weekends]<br>

3. time on weekends<br>
    6-10<br>
    10-16<br>
    16-21



In [2]:
import numpy as np
import pandas as pd
import pickle
import networkx as nx
import datetime as dt


import community as community_louvain

import folium
from folium import plugins
import matplotlib.cm as cm
import matplotlib.pyplot as plt


## Location EDA

- sensor location visualization
- community detection based on location

In [11]:
location = pd.read_csv('graph_sensor_locations_la.csv',header = 0 )
location.columns=['index','sensor_id','latitude','longitude']
location = location[['sensor_id','latitude','longitude']]
location

,sensor_id,latitude,longitude
0,773869,34.15497,-118.31829
1,767541,34.11621,-118.23799
2,767542,34.11641,-118.23819
3,717447,34.07248,-118.26772
4,717446,34.07142,-118.26572
...,...,...,...
202,717592,34.14604,-118.22430
203,717595,34.14163,-118.18290
204,772168,34.16542,-118.47985
205,718141,34.15133,-118.37456


In [21]:
map_osm = folium.Map(location=[34.14604,-118.22430], zoom_start=11)


for i in location.index:
    # 행 우선 접근 방식으로 값 추출하기
    name = location.loc[i, 'sensor_id']
    lat = location.loc[i, 'latitude']
    lng = location.loc[i, 'longitude']
    
    # 추출한 정보를 지도에 표시
    marker = folium.Marker([lat,lng], popup=name,  icon=plugins.BeautifyIcon(border_width=6 ,icon_shape = 'doughnut', border_color=colors[c]))
    marker.add_to(map_osm)

map_osm

In [16]:
### Read adj_matrix (DCRNN>data)
with open('./adj_mx_la_distance.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    adj_bay = u.load()
adj_bay = np.array(adj_bay)[-1]
adj_bay

G = nx.Graph(adj_bay)
# compute the best partition
partition = community_louvain.best_partition(G)

'''
# draw the graph
pos = nx.spring_layout(G)
# color the nodes according to their partition
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40,
                       cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()
'''


set(partition.values()) ## 25 communities

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [17]:
colors = ['orange', 'blue', 'lightblue', 'lightred', 'beige', 'darkred', 'lightgreen', 'darkblue', 'black', 'darkgreen', 'pink', 'cadetblue', 'white', 'green', 'gray', 'darkpurple', 'purple', 'lightgray', 'red']
len(colors)

19

### Community detection based on distance

It it the result of community detection by distance based adjacency matrix.<br>
The communities are seperated depend on intersection, because in the data, there is not distance between sensors beyond intersection.

In [45]:


map_osm = folium.Map(location=[34.14604,-118.22430], zoom_start=11)


for i in location.index:
    # 행 우선 접근 방식으로 값 추출하기
    name = location.loc[i, 'sensor_id']
    lat = location.loc[i, 'latitude']
    lng = location.loc[i, 'longitude']
    c = partition[i]

    if c <= 18:
        marker = folium.Marker([lat,lng], popup=name,  icon=folium.Icon(color=colors[c]))
        marker.add_to(map_osm)


map_osm

## Comumunity detection by Speed

In [22]:
'''
### Read speed data to do prediction task
speed_bay = pd.read_hdf('./pems-bay.h5')
speed_bay

'''


"\n### Read speed data to do prediction task\nspeed_bay = pd.read_hdf('./pems-bay.h5')\nspeed_bay\n\n"

In [24]:

speed_bay = pd.read_csv("la_speed.csv")
speed_bay

,Unnamed: 0,773869,767541,767542,717447,717446,717445,773062,767620,737529,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
0,2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125000,67.125000,59.625000,...,45.625000,65.500000,64.500000,66.428571,66.875000,59.375000,69.000000,59.250000,69.000000,61.875000
1,2012-03-01 00:05:00,62.666667,68.555556,65.444444,62.444444,64.444444,68.111111,65.000000,65.000000,57.444444,...,50.666667,69.875000,66.666667,58.555556,62.000000,61.111111,64.444444,55.888889,68.444444,62.875000
2,2012-03-01 00:10:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500000,64.250000,63.875000,...,44.125000,69.000000,56.500000,59.250000,68.125000,62.500000,65.625000,61.375000,69.857143,62.000000
3,2012-03-01 00:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2012-03-01 00:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34267,2012-06-27 23:35:00,65.000000,65.888889,68.555556,61.666667,0.000000,54.555556,62.444444,63.333333,59.222222,...,52.888889,69.000000,65.111111,55.666667,66.333333,62.444444,66.777778,64.888889,69.666667,62.333333
34268,2012-06-27 23:40:00,61.375000,65.625000,66.500000,62.750000,0.000000,50.500000,62.000000,67.000000,65.250000,...,54.000000,69.250000,60.125000,60.500000,67.250000,59.375000,66.000000,61.250000,69.000000,62.000000
34269,2012-06-27 23:45:00,67.000000,59.666667,69.555556,61.000000,0.000000,44.777778,64.222222,63.777778,59.777778,...,51.333333,67.888889,64.333333,57.000000,66.000000,62.666667,68.666667,63.333333,67.444444,61.222222
34270,2012-06-27 23:50:00,66.750000,62.250000,66.000000,59.625000,0.000000,53.000000,64.285714,64.125000,60.875000,...,51.125000,69.375000,61.625000,60.500000,65.625000,66.375000,69.500000,63.000000,67.875000,63.500000


### add day_name

In [25]:
speed_bay['Unnamed: 0'] = pd.to_datetime(speed_bay['Unnamed: 0'])
speed_bay['w_name'] = speed_bay['Unnamed: 0'].dt.day_name()
speed_bay.head()

,Unnamed: 0,773869,767541,767542,717447,717446,717445,773062,767620,737529,...,769372,774204,769806,717590,717592,717595,772168,718141,769373,w_name
0,2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125,67.125,59.625000,...,65.500,64.500000,66.428571,66.875,59.375000,69.000000,59.250000,69.000000,61.875,Thursday
1,2012-03-01 00:05:00,62.666667,68.555556,65.444444,62.444444,64.444444,68.111111,65.000,65.000,57.444444,...,69.875,66.666667,58.555556,62.000,61.111111,64.444444,55.888889,68.444444,62.875,Thursday
2,2012-03-01 00:10:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500,64.250,63.875000,...,69.000,56.500000,59.250000,68.125,62.500000,65.625000,61.375000,69.857143,62.000,Thursday
3,2012-03-01 00:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,...,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,Thursday
4,2012-03-01 00:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,...,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,Thursday


### seperate data by time

6-10,10-16,16-21

In [26]:
date_time = list(speed_bay['Unnamed: 0'])

In [27]:
six_to_10 = []
ten_to_16 =[]
sixteens_to_21 = []

for i in range(len(date_time)):
    if date_time[i].hour in range(6,10):
        six_to_10.append(speed_bay.iloc[i])
        
    elif speed_bay['Unnamed: 0'][i].hour in range(10,16):
        ten_to_16.append(speed_bay.iloc[i])
        
    elif speed_bay['Unnamed: 0'][i].hour in range(16,21):
        sixteens_to_21.append(speed_bay.iloc[i])
        
        
six_to_10 = pd.DataFrame(six_to_10)
ten_to_16 = pd.DataFrame(ten_to_16)
sixteens_to_21 = pd.DataFrame(sixteens_to_21)

### weekdays and weekends

In [28]:
six_to_10_S = six_to_10[(six_to_10['w_name'] == 'Sunday')| (six_to_10['w_name'] == 'Saturday')]
six_to_10_W = six_to_10[(six_to_10['w_name']!= 'Sunday') & (six_to_10['w_name'] != 'Saturday')]


ten_to_16_S = ten_to_16[(ten_to_16['w_name'] == 'Sunday')| (ten_to_16['w_name'] == 'Saturday')]
ten_to_16_W = ten_to_16[(ten_to_16['w_name']!= 'Sunday') & (ten_to_16['w_name'] != 'Saturday')]

sixteens_to_21_S = sixteens_to_21[(sixteens_to_21['w_name'] == 'Sunday')| (sixteens_to_21['w_name'] == 'Saturday')]
sixteens_to_21_W = sixteens_to_21[(sixteens_to_21['w_name']!= 'Sunday') & (sixteens_to_21['w_name'] != 'Saturday')]


In [29]:
six_to_10_W

,Unnamed: 0,773869,767541,767542,717447,717446,717445,773062,767620,737529,...,769372,774204,769806,717590,717592,717595,772168,718141,769373,w_name
72,2012-03-01 06:00:00,66.500000,60.000000,67.875000,54.375000,64.875000,64.125000,68.375000,62.875000,49.500000,...,66.500000,61.375000,63.375000,66.125000,63.875000,66.375000,56.625000,67.500000,57.000000,Thursday
73,2012-03-01 06:05:00,64.285714,61.714286,65.571429,56.142857,64.428571,63.142857,64.000000,65.000000,42.571429,...,68.000000,63.571429,61.428571,64.714286,62.285714,63.714286,56.142857,64.857143,51.285714,Thursday
74,2012-03-01 06:10:00,65.875000,61.250000,67.000000,54.750000,62.000000,61.000000,67.250000,62.500000,30.500000,...,67.625000,60.750000,63.250000,64.125000,60.875000,63.625000,50.625000,66.500000,52.250000,Thursday
75,2012-03-01 06:15:00,67.777778,64.000000,67.777778,53.333333,63.222222,61.777778,67.222222,65.000000,13.888889,...,67.666667,60.666667,66.666667,65.000000,64.222222,67.333333,41.000000,68.666667,53.333333,Thursday
76,2012-03-01 06:20:00,67.125000,62.375000,68.000000,54.750000,62.250000,63.000000,65.750000,64.000000,19.875000,...,67.250000,61.375000,66.000000,65.625000,61.625000,67.625000,38.000000,65.875000,54.250000,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34099,2012-06-27 09:35:00,64.250000,64.375000,44.000000,47.750000,37.125000,58.500000,62.125000,62.625000,60.875000,...,68.125000,59.500000,66.285714,63.500000,64.500000,66.875000,43.166667,63.875000,64.000000,Wednesday
34100,2012-06-27 09:40:00,63.375000,65.250000,64.000000,51.500000,29.000000,57.000000,63.375000,66.000000,61.875000,...,66.750000,60.625000,65.428571,66.500000,62.500000,66.500000,47.857143,68.714286,62.625000,Wednesday
34101,2012-06-27 09:45:00,63.000000,64.222222,65.555556,50.777778,30.333333,56.111111,62.000000,64.777778,62.777778,...,67.777778,57.777778,64.666667,65.888889,64.333333,68.444444,52.111111,66.333333,61.555556,Wednesday
34102,2012-06-27 09:50:00,63.750000,64.125000,63.125000,51.000000,35.875000,55.875000,60.250000,61.250000,64.000000,...,66.625000,60.625000,65.000000,66.750000,63.000000,66.750000,45.833333,67.000000,64.250000,Wednesday


### make adjacent matrix

- for mean of sensor's speed
- by using gaussian kernels

In [32]:
def Gaussian(x,z,sigma):
    return np.exp((-(np.linalg.norm(x-z)**2))/(2*sigma**2))

def GaussianMatrix(X,sigma):
    row,col=X.shape
    GassMatrix=np.zeros(shape=(row,row))
    X=np.asarray(X)
    i=0
    for v_i in X:
        j=0
        for v_j in X:
            GassMatrix[i,j]=Gaussian(v_i.T,v_j.T,sigma)
            j+=1
        i+=1
    return GassMatrix

def adj_mat_speed(data):
    mean = pd.DataFrame(data.mean(axis = 0))
    adj_mat = GaussianMatrix(mean,sigma = 1)
    
    return adj_mat

In [46]:
def visualization_community(data,location):
    
    G = nx.Graph(adj_mat_speed(data))
    # compute the best partition
    partition = community_louvain.best_partition(G)
    
    stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])
    stat['community'] = partition.values()
    report = stat.groupby(stat['community']).describe()
    report.index = colors[:len(report)]
    print(report)
    
    
    map_osm = folium.Map(location=[34.14604,-118.22430], zoom_start=11)

    for i in location.index:
        # 행 우선 접근 방식으로 값 추출하기
        name = location.loc[i, 'sensor_id']
        lat = location.loc[i, 'latitude']
        lng = location.loc[i, 'longitude']
        c = partition[i]
        
        # can visualize 18th community.(because of limiation of Marker)
        if c <= 18:
            marker = folium.Marker([lat,lng], popup=name, icon=folium.Icon(color=colors[c]))
            marker.add_to(map_osm)

    return map_osm




map_osm = folium.Map(location=[34.14604,-118.22430], zoom_start=11)


### 6-10 weekdays




In [47]:
visualization_community(six_to_10_W,location)

<ipython-input-32-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
<ipython-input-46-d756461ffd80>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])


           AvgSpeed                                                        \
              count       mean       std        min        25%        50%   
orange         23.0  57.540744  0.548280  56.640843  57.137809  57.577440   
blue           27.0  59.751346  0.506022  58.652105  59.452332  59.720138   
lightblue      35.0  40.880575  2.249888  36.623140  39.355968  40.751777   
lightred       25.0  48.669354  1.426962  46.156457  47.655583  48.787394   
beige           4.0  16.305825  1.635588  14.636203  15.325941  16.068280   
darkred        46.0  53.748457  1.362475  51.650251  52.680003  53.643150   
lightgreen      6.0  23.793423  1.826106  21.931922  22.139205  23.804977   
darkblue       21.0  31.777304  2.294841  28.342454  30.129343  31.081137   
black          20.0  62.061479  0.553702  61.001331  61.770389  62.081503   

                                  
                  75%        max  
orange      57.876783  58.443627  
blue        60.023444  60.881986  
lightblue   

### 6-10 weekends

In [48]:
visualization_community(six_to_10_S,location)

<ipython-input-32-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
<ipython-input-46-d756461ffd80>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])


          AvgSpeed                                                        \
             count       mean       std        min        25%        50%   
orange        58.0  56.347078  0.718268  55.000018  55.806669  56.436174   
blue          48.0  58.138944  0.459365  57.408585  57.664216  58.141479   
lightblue     52.0  59.926136  0.524953  59.026759  59.552554  59.940195   
lightred      42.0  52.459975  1.789348  47.846707  51.169597  53.007271   
beige          2.0  30.708850  1.061513  29.958247  30.333549  30.708850   
darkred        5.0  41.552843  2.632719  37.672512  40.878780  41.512896   

                                 
                 75%        max  
orange     56.971165  57.345920  
blue       58.561334  58.851637  
lightblue  60.218487  61.326502  
lightred   53.901532  54.832688  
beige      31.084152  31.459454  
darkred    42.939535  44.760491  


### 6-10 all days

In [49]:
visualization_community(six_to_10,location)

<ipython-input-32-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
<ipython-input-46-d756461ffd80>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])


           AvgSpeed                                                        \
              count       mean       std        min        25%        50%   
orange         20.0  56.842065  0.554894  55.776089  56.410776  56.868937   
blue           38.0  59.117252  0.763234  57.882665  58.477660  59.191233   
lightblue      28.0  50.611580  1.136099  48.378694  49.717234  50.668749   
lightred       30.0  45.444694  1.393191  42.644787  44.373436  45.648448   
beige          14.0  30.371036  3.379725  23.942530  27.844433  31.197149   
darkred        39.0  54.130851  0.774397  52.683169  53.539339  54.148448   
lightgreen     18.0  61.524280  0.445128  60.882612  61.257482  61.469701   
darkblue       20.0  39.347565  1.743382  36.855777  37.430717  39.326376   

                                  
                  75%        max  
orange      57.234873  57.722541  
blue        59.620151  60.481891  
lightblue   51.535165  52.200514  
lightred    46.736893  47.465394  
beige       33.1212

In [50]:
six_to_10_S = six_to_10[(six_to_10['w_name'] == 'Sunday')| (six_to_10['w_name'] == 'Saturday')]
six_to_10_W = six_to_10[(six_to_10['w_name']!= 'Sunday') & (six_to_10['w_name'] != 'Saturday')]


ten_to_16_S = ten_to_16[(ten_to_16['w_name'] == 'Sunday')| (ten_to_16['w_name'] == 'Saturday')]
ten_to_16_W = ten_to_16[(ten_to_16['w_name']!= 'Sunday') & (ten_to_16['w_name'] != 'Saturday')]

sixteens_to_21_S = sixteens_to_21[(sixteens_to_21['w_name'] == 'Sunday')| (sixteens_to_21['w_name'] == 'Saturday')]
sixteens_to_21_W = sixteens_to_21[(sixteens_to_21['w_name']!= 'Sunday') & (sixteens_to_21['w_name'] != 'Saturday')]


### 10-16 weekdays

In [51]:
visualization_community(ten_to_16_W,location)

<ipython-input-32-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
<ipython-input-46-d756461ffd80>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])


           AvgSpeed                                                        \
              count       mean       std        min        25%        50%   
orange         49.0  56.832170  0.915137  55.024553  56.271461  56.984271   
blue           43.0  60.732233  1.417745  58.538106  59.528465  60.721578   
lightblue      30.0  47.370688  1.283035  44.971921  46.378906  47.498603   
lightred       27.0  40.958007  3.014648  34.183355  39.594062  41.249980   
beige          49.0  52.220720  1.188545  50.132861  51.459720  52.134772   
darkred         2.0  21.894624  0.948811  21.223713  21.559168  21.894624   
lightgreen      7.0  27.683158  2.103517  25.514418  26.301839  26.894953   

                                  
                  75%        max  
orange      57.564511  58.255043  
blue        61.698666  63.409903  
lightblue   48.388887  49.563592  
lightred    43.383118  44.691340  
beige       53.230263  54.279707  
darkred     22.230079  22.565534  
lightgreen  28.805863  31.

### 10-16 weekends

In [52]:
visualization_community(ten_to_16_S,location)

<ipython-input-32-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
<ipython-input-46-d756461ffd80>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])


           AvgSpeed                                                        \
              count       mean       std        min        25%        50%   
orange         45.0  59.474126  0.855468  58.002211  58.765801  59.485525   
blue           40.0  62.984825  0.919292  61.306755  62.204934  63.117880   
lightblue      23.0  47.495406  1.414360  44.604521  46.758159  47.669051   
lightred       19.0  37.784006  4.015240  30.999167  34.846129  37.910130   
beige          40.0  52.582792  1.327816  50.103650  51.632109  52.739619   
darkred        34.0  56.391300  0.754577  55.107073  55.751612  56.378539   
lightgreen      6.0  26.229947  1.758866  23.265507  25.704914  26.290621   

                                  
                  75%        max  
orange      60.192337  61.105340  
blue        63.775220  64.326445  
lightblue   48.593553  49.490949  
lightred    41.375877  43.274867  
beige       53.707102  54.684774  
darkred     56.967513  57.585401  
lightgreen  27.456062  28.

### 10-16 all days

In [53]:
visualization_community(ten_to_16,location)

<ipython-input-32-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
<ipython-input-46-d756461ffd80>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])


           AvgSpeed                                                        \
              count       mean       std        min        25%        50%   
orange         48.0  58.035945  0.796694  56.429850  57.362949  58.022730   
blue           37.0  61.430021  1.242632  59.563887  60.526817  61.277667   
lightblue      40.0  48.162641  1.539026  45.087715  47.013757  48.309097   
lightred        5.0  35.907342  0.904661  34.605098  35.786543  35.814663   
beige          51.0  53.474481  1.468848  50.735033  52.182080  53.536626   
darkred         9.0  26.826713  3.018565  21.807083  25.807858  27.093502   
lightgreen     17.0  42.294007  1.266366  39.814202  41.589999  42.222636   

                                  
                  75%        max  
orange      58.610568  59.451040  
blue        62.300510  63.671772  
lightblue   49.138892  50.499834  
lightred    36.214692  37.115712  
beige       54.475598  56.168492  
darkred     28.067586  31.295989  
lightgreen  42.702053  44.

### 16-21 weekdays

In [54]:
visualization_community(sixteens_to_21_W,location)

<ipython-input-32-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
<ipython-input-46-d756461ffd80>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])


           AvgSpeed                                                        \
              count       mean       std        min        25%        50%   
orange         39.0  43.289793  1.533730  40.767828  42.030408  43.542887   
blue           31.0  60.922198  1.159576  59.039984  59.978609  61.199652   
lightblue      23.0  64.822729  0.837802  63.542622  64.215127  64.873811   
lightred       26.0  38.339671  1.256857  35.452215  37.551951  38.386633   
beige          14.0  47.194911  0.612905  46.241314  46.634322  47.393597   
darkred        30.0  56.758550  1.274270  54.760416  55.823629  56.661419   
lightgreen      3.0  24.631694  1.392884  23.026799  24.184760  25.342722   
darkblue       31.0  51.222698  1.425934  48.762493  50.238730  51.172988   
black          10.0  32.143662  1.581205  29.702374  31.145611  32.125154   

                                  
                  75%        max  
orange      44.277432  45.823935  
blue        61.795463  62.792911  
lightblue   

### 16-21 weekends

In [55]:
visualization_community(sixteens_to_21_S,location)

<ipython-input-32-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
<ipython-input-46-d756461ffd80>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])


          AvgSpeed                                                        \
             count       mean       std        min        25%        50%   
orange        60.0  56.613083  1.223765  54.258828  55.495891  56.883223   
blue          55.0  63.328768  1.104686  61.487170  62.333760  63.337721   
lightblue     27.0  50.700738  2.019349  46.152694  49.361124  50.945899   
lightred      20.0  40.628639  2.931500  34.538024  38.396935  41.441457   
beige         40.0  59.998082  0.504093  58.932114  59.698311  60.002272   
darkred        5.0  27.088619  1.553771  25.594960  26.004145  26.361588   

                                 
                 75%        max  
orange     57.479682  58.652395  
blue       64.279696  65.268198  
lightblue  52.488027  53.615504  
lightred   42.891664  45.155125  
beige      60.285956  60.856071  
darkred    28.381288  29.101116  


### 16-21 all days

In [56]:
visualization_community(sixteens_to_21,location)

<ipython-input-32-e222ea16b736>:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean = pd.DataFrame(data.mean(axis = 0))
<ipython-input-46-d756461ffd80>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stat = pd.DataFrame(data.mean(axis = 0),columns = ['AvgSpeed'])


           AvgSpeed                                                        \
              count       mean       std        min        25%        50%   
orange         35.0  48.386470  1.114566  46.595054  47.559564  48.468529   
blue           27.0  61.528468  0.841189  60.127810  60.910083  61.491259   
lightblue      21.0  64.680967  0.726040  63.432540  63.976051  64.833322   
lightred       30.0  44.410754  1.309912  41.724736  43.189460  44.765421   
beige          16.0  39.328975  1.198048  37.095446  38.557455  39.673151   
darkred        25.0  55.936002  0.802407  54.325977  55.383607  56.014589   
lightgreen     19.0  58.718727  0.661676  57.571151  58.308729  58.964675   
darkblue        3.0  25.579209  1.385814  23.979596  25.160540  26.341484   
black          24.0  52.436623  0.669983  50.998446  51.840733  52.588043   
darkgreen       7.0  32.297930  1.303798  30.585073  31.225142  32.512765   

                                  
                  75%        max  
orang